# This sample was inspired by [08-native-function-inline.ipynb](https://github.com/microsoft/semantic-kernel/blob/main/python/notebooks/08-native-function-inline.ipynb)

## [How does Python SK compare to the C# version of Semantic Kernel?](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)
- The two SDKs are compatible and at the core they follow the same design principles.
- Some features are still available only in the C# version, and being ported.
- Refer to the [FEATURE MATRIX](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) doc to see where things stand in matching the features and functionality of the main SK branch.
- Over time there will be some features available only in the Python version, and others only in the C# version, for example adapters to external services, scientific libraries, etc.
<br/>
- Documentation
 - [Get Started with Semantic Kernel](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)

Inspired to
 - https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example59_OpenAIFunctionCalling.cs
 - https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

In [9]:
# Create "Native" (aka "Core") Plugins that contain Functions

from semantic_kernel.plugin_definition import kernel_function

class RandomNumberPlugin:    
    """
    Description: Generate some random numbers.
    """

    @kernel_function(
        description="Generate a random number between 3 and x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3 and <input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        import random

        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

    @kernel_function(
        description="Generate a random number between x and three",
        name="GenerateNumberThreeOrLower",
    )
    def generate_number_three_or_lower(self, input: str) -> str:
        """
        Generate a number between <input> and 3
        Example:
            "-4" => rand(-4,3)
        Args:
            input -- The lower limit for the random number generation
        Returns:
            int value
        """
        import random
        
        try:
            return str(random.randint(int(input),3))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e
            
            
class CatsPlugin:
    """
    Description: Given a cat name, returns some information.
    """

    @kernel_function(
        description="Given a cat name, returns its age.",
        name="CatAge",
    )
    def cat_age(self, cat_name: str) -> str:
        """
        Given a cat name, returns its age.
        """
        try:
            return str(len(cat_name))
        except ValueError as e:
            print(f"Invalid input {cat_name}")
            raise e
            
    @kernel_function(
        description="Given a cat name, returns its birthday.",
        name="CatBirthday",
    )
    def cat_birthday(self, cat_name: str) -> str:
        """
        Given a cat name, returns its birthday.
        """        
        from datetime import datetime
        from dateutil.relativedelta import relativedelta
        
        today_str  = datetime.today().strftime("%Y-%m-%d")
        today_date = datetime.strptime(today_str,"%Y-%m-%d")

        try:
            cat_birthday_str = today_date.strftime(f"{2000+len(cat_name)}-%m-%d")
            cat_birthday_date = datetime.strptime(cat_birthday_str,"%Y-%m-%d") # error raised when 29/2 in non-leap years
        except ValueError as e:
            cat_birthday_str = (today_date + relativedelta(days=1)).strftime(f"{2000+len(cat_name)}-%m-%d") # take March 1st

        return cat_birthday_str

            
# test RandomNumberPlugin class / generate_number_three_or_higher function
my_randomnumber_plugin = RandomNumberPlugin()  # Create an instance
random_number = my_randomnumber_plugin.generate_number_three_or_higher(100) # Call the method
print(f"result: {random_number}") # Assert the result

# test CatsPlugin class / cat_age function
my_cat_plugin = CatsPlugin()  # Create an instance
age = my_cat_plugin.cat_birthday("Molly") # Call the method
print(f"age: {age}") # Assert the result

result: 30
age: 2005-02-05


In [7]:
# create the kernel

import semantic_kernel as sk
kernel = sk.Kernel()

In [8]:
# import plugins into the kernel, together with all their (native) functions

kernel.import_plugin(plugin_instance=RandomNumberPlugin(), plugin_name="Random_Number_Plugin")
kernel.import_plugin(plugin_instance=CatsPlugin(), plugin_name="Cats_Plugin")

print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

Here are my plugins imported into the kernel:
data={'random_number_plugin': {'generatenumberthreeorhigher': KernelFunction(), 'generatenumberthreeorlower': KernelFunction()}, 'cats_plugin': {'catage': KernelFunction(), 'catbirthday': KernelFunction()}}


In [10]:
# test plugin functions after extracting them from the kernel

print(kernel.plugins.get_function(plugin_name="Random_Number_Plugin", function_name="generatenumberthreeorhigher")("20"))
print(kernel.plugins.get_function(plugin_name="Cats_Plugin", function_name="CatBirthday")("Tom"))
print(kernel.plugins.get_function(plugin_name="Cats_Plugin", function_name="CatBirthday")("Cleopatra"))

14
2003-02-05
2009-02-05


In [11]:
# another way to test Cats_Birthday function after extracting it from the kenel

cb_function = kernel.plugins.get_function(plugin_name="Cats_Plugin", function_name="CatBirthday")
answer = cb_function.invoke("Cleopatra")
print (answer)

2009-02-05


In [12]:
# We haven't used OpenAI yet
# So we create the SK "chat completion connector" to the Azure OpenAI service

from dotenv import load_dotenv
load_dotenv("credentials_my.env")

import semantic_kernel.connectors.ai.open_ai as sk_oai

chat_connector = sk_oai.AzureChatCompletion(
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    deployment_name=os.environ['GPT35TURBO-0613-4k'], # ['GPT4-1106-128k'],
    endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)
print(f"This is my AzureChatCompletion connector:\n{chat_connector}")

This is my AzureChatCompletion connector:
ai_model_id='gpt35turbo-0613-4k' client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7f4e4c1718b0> ai_model_type=<OpenAIModelTypes.CHAT: 'chat'> prompt_tokens=0 completion_tokens=0 total_tokens=0


In [13]:
# add the openAI completion connector to the kernel, choosing a name to identify it within the kernel
# after this instruction, the kernel has got it service_id that you can get with kernel.all_chat_services()

kernel.add_chat_service(
    "mauromi_chatgpt", # unique name to be registered with the kernel
    chat_connector
)
print(f"Here are all chat services registered with this kernel: {kernel.all_chat_services()}")

Here are all chat services registered with this kernel: ['mauromi_chatgpt']


In [14]:
# Now that we have the chat service registered with the kernel, we can create and run a semantic function IN-LINE
# The semantic function is added to the list of kernel semantic functions
# We may also add an existing SEMANTIC plugin with kernel.import_semantic_plugin_from_directory(plugins_directory, "FunPlugin")
# We may also add an existing NATIVE plugin with import_native_plugin_from_directory(plugins_directory, "FunPlugin")
# https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

sk_prompt = "{{$input}}"

my_semantic_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    plugin_name="GenericSemanticPlugin",
    function_name="GenericSemanticFunction",
    description="Generic Semantic Function in a Generic Semantic Plugin",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

Here are my plugins imported into the kernel:
data={'random_number_plugin': {'generatenumberthreeorhigher': KernelFunction(), 'generatenumberthreeorlower': KernelFunction()}, 'cats_plugin': {'catage': KernelFunction(), 'catbirthday': KernelFunction()}, 'genericsemanticplugin': {'genericsemanticfunction': KernelFunction()}}


In [15]:
# Call the semantic function directly, without extracting it from the kernel

context = kernel.create_new_context()
context.variables["input"] = "Who is Joe Biden's Wife?"
print(f"Context before: {context}")
print(f"Result: {my_semantic_function.invoke(context=context)}")
print(f"Context after: {context}")

Context before: Who is Joe Biden's Wife?
Result: Joe Biden's wife is Jill Biden.
Context after: Joe Biden's wife is Jill Biden.


In [ ]:
# Same result as previous cell, but in this case we extract the semantic funt from the kernel and THEN we invoke it

context = kernel.create_new_context()
context.variables["input"] = "Who is Joe Biden's Wife?"
sf = kernel.plugins.get_function(plugin_name="GenericSemanticPlugin", function_name="GenericSemanticFunction")
print(f"Context before: {context}")
print(sf.invoke(context=context))
print(f"Context after: {context}")

In [16]:
import json

all_functions = kernel.plugins.get_functions_view()
json_object = json.loads(all_functions.json())
json_formatted_str = json.dumps(json_object, indent=2)

print(json_formatted_str)

{
  "semantic_functions": {
    "GenericSemanticPlugin": [
      {
        "name": "GenericSemanticFunction",
        "plugin_name": "GenericSemanticPlugin",
        "description": "Generic Semantic Function in a Generic Semantic Plugin",
        "is_semantic": true,
        "parameters": [
          {
            "name": "input",
            "description": "",
            "default_value": "",
            "type_": "string",
            "required": false
          }
        ],
        "is_asynchronous": true
      }
    ]
  },
  "native_functions": {
    "Random_Number_Plugin": [
      {
        "name": "GenerateNumberThreeOrHigher",
        "plugin_name": "Random_Number_Plugin",
        "description": "Generate a random number between 3 and x",
        "is_semantic": false,
        "parameters": [],
        "is_asynchronous": true
      },
      {
        "name": "GenerateNumberThreeOrLower",
        "plugin_name": "Random_Number_Plugin",
        "description": "Generate a random numbe

Bad pipe message: %s [b'F\x96i_`k\x95M\xa3\x0eNk\xd7xw\xce9\xcb\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00']
Bad pipe message: %s [b"j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\xc0\x11\xc0\x08\xc0\x12\x00\x16\x00\x13\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00\xc0\x00<\x00\xba\x005\x00\x84\x00/\x00\x96\x00A\x00\x05\x00\n\x00\xff\x01"]
Bad pipe message: %s [b'\xb6\x1fwY\x93}\x96\xad~\xfe\xd1\x06\x05\xef\xfb\xaf\x03\x1b\x00\x00>\xc0']
Bad pipe message: %s [b'\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t']
Bad pipe message: %s [b'\xaa\xfa\xc0\xb7\x04\x84y\x1f\x02\xbbI+\\\x11m\xa9\xe7\x97\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0

# https://youtu.be/1e8GOdTPHC4

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flights",            
            "description": "returns the number of flights between two dates",
            "parameters": {
                "type": "object",
                "properties": {
                    "date_1": {"type": "string", "description": "the first date"},
                    "date_2": {"type": "string", "description": "the second date"}
                }
            },
            "required": ["date_1", "date_2"]
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_days_difference",            
            "description": "useful when you need to calculate the absolute number of days between two dates",
            "parameters": {
                "type": "object",
                "properties": {
                    "date_1": {"type": "string", "description": "the first date"},
                    "date_2": {"type": "string", "description": "the second date"}
                }
            },
            "required": ["date_1", "date_2"]
        }
    },
    {
        "type": "function",
        "function": {
            "name": "elevation_to_power",
            "description": "calculate the mathematical result of a number elevated to the power of another number",
            "parameters": {
                "type": "object",
                "properties": {
                    "base": {"type": "string", "description": "the base number"},
                    "power": {"type": "string", "description": "the power to elevate the base number to"}
                }
            },
            "required": ["base", "power"]
        }
    },
    {
        "type": "function",
        "function": {
            "name": "web_search",            
            "description": "useful for when you need to answer questions about current events",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "the text string to search on the WEB"}
                }
            },
            "required": ["question"]
        }
    },
    {
        "type": "function",
        "function": {
            "name": "current_year",            
            "description": "returns the current year",
            "parameters": {
                "type": "object",
                "properties": {                    
                }
            },
            "required": []
        }
    },
    {
        "type": "function",
        "function": {
            "name": "current_date",            
            "description": "returns the current date",
            "parameters": {
                "type": "object",
                "properties": {
                    
                }
            },
            "required": []
        }
    },
    {
        "type": "function",
        "function": {
            "name": "my_cat_born_date",            
            "description": "returns my cat's born date",
            "parameters": {
                "type": "object",
                "properties": {
                    
                }
            },
            "required": []
        }
    },
    {
        "type": "function",
        "function": {
            "name": "custom_calculator",            
            "description": "useful for when you need to answer questions about math",
            "parameters": {
                "type": "object",
                "properties": {
                    "mathematical_question": {"type": "string", "description": "the text string to search on the WEB"}
                }
            },
            "required": ["mathematical_question"]
        }
    }
]

In [ ]:
# prompt template config that contains the definition and description of each function

prompt_config = sk.PromptTemplateConfig.from_execution_settings(
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
    tool_choice="auto",
    tools=tools,
)
prompt_config

In [ ]:
# Let's check that we have the chat completion service

kernel.get_chat_service_service_id()

In [ ]:
prompt_template = sk.ChatPromptTemplate[sk_oai.models.chat.open_ai_chat_message.OpenAIChatMessage](
    "{{$user_input}}", kernel.prompt_template_engine, prompt_config
)
prompt_template

In [ ]:
prompt_template.add_user_message("Hi there, who are you?")

In [ ]:
function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)

In [ ]:
function_config

In [ ]:
class Test:
    @kernel_function(
        description="Get light status",
        name="getlightstatus",
    )
    def getlightstatus(self) -> str:
        return "off"
    
chat_function = kernel.register_native_function(plugin_name="Testzzzz", kernel_function="getLightStatus")



In [ ]:
import json

all_functions = kernel.plugins.get_functions_view()
json_object = json.loads(all_functions.json())
json_formatted_str = json.dumps(json_object, indent=2)

print(json_formatted_str)

In [ ]:
context = kernel.create_new_context()
context.variables["user_input"] = "How many flights are there between Christmas 2020 and Easter 2021?"

    
response = chat_function.invoke_async(context=context, functions=tools)
response = chat_function.invoke()
response

In [ ]:
context.objects